# Remote actions and variables

Changing the state of remote resources


---
- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
---




**Prerequisites**:
- A Grid'5000 account
- A working EnOSlib environment and Jupyter (not included in EnOSlib dependencies, but `pip install jupyterlab` will install it)




In [1]:
import enoslib as en

# get some logging info
import logging
logging.basicConfig(level=logging.INFO)

## Setup on Grid'5000

### Describing the resources

For the purpose of the tutorial we'll reserve 2 nodes in the production environment.

First we build a configuration object describing the wanted resources: `machines` and `networks`.

In [2]:
network = en.G5kNetworkConf(id="n1", type="prod", roles=["my_network"], site="rennes")

conf = (
    en.G5kConf.from_settings(job_type="allow_classic_ssh", job_name="rsd-01")
    .add_network_conf(network)
    .add_machine(
        roles=["control"], cluster="parasilo", nodes=1, primary_network=network
    )
    .add_machine(
        roles=["compute"],
        cluster="parasilo",
        nodes=1,
        primary_network=network,
    )
    .finalize()
)
conf

Conf@0x7fce1b88add0
{
    "dhcp": true,
    "force_deploy": false,
    "env_name": "debian10-x64-nfs",
    "job_name": "rsd-01",
    "job_type": "allow_classic_ssh",
    "key": "/home/msimonin/.ssh/id_rsa.pub",
    "queue": "default",
    "walltime": "02:00:00",
    "resources": {
        "machines": [
            {
                "roles": [
                    "control"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            },
            {
                "roles": [
                    "compute"
                ],
                "primary_network": "n1",
                "secondary_networks": [],
                "cluster": "parasilo",
                "nodes": 1
            }
        ],
        "networks": [
            {
                "id": "n1",
                "type": "prod",
                "roles": [
                    "my_network"
                ],
                "site": "rennes"
            }
        ]
    }
}

### Reserving the resources

We can pass the `Configuration` object to the `G5k` provider. The provider is responsible to turn the abstract resource description into a et concrete resources.

In [3]:
provider = en.G5k(conf)
roles, networks = provider.init()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Submitting {'name': 'rsd-01', 'types': ['allow_classic_ssh'], 'resources': "{cluster='parasilo'}/nodes=1+{cluster='parasilo'}/nodes=1,walltime=02:00:00", 'command': 'sleep 31536000', 'queue': 'default'} on rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1816344 on rennes [2021-08-27 10:00:58]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1816344 on rennes [2021-08-27 10:00:58]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Runni

Inspecting the ressources we own for the experiment's lifetime:

- roles: this is somehow a dictionnary whose keys are the role names and the associated values are the corresponding list of hosts
- networks: similar to roles bu£t for networks

In [4]:
roles

{'control': [Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-22.rennes.grid5000.fr', alias='parasilo-22.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [5]:
# list of host on a given role
roles["control"]

[Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]

In [6]:
# a single host
roles["control"][0]

Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())

In [7]:
networks

{'my_network': [<enoslib.infra.enos_g5k.objects.G5kEnosProd4Network object at 0x7fce1b8a6350>, <enoslib.infra.enos_g5k.objects.G5kEnosProd6Network object at 0x7fce1ae0a110>]}

`provider.init` is idempotent. In the Grid'5000 case, you can call it several time in a row. The same reservation will reloaded and the roles and networks will be the same.

In [8]:
roles, networks = provider.init()
roles

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1816344 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Waiting for 1816344 on rennes [2021-08-27 10:00:58]
INFO:enoslib.infra.enos_g5k.g5k_api_utils:All jobs are Running !


{'control': [Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())], 'compute': [Host(address='parasilo-22.rennes.grid5000.fr', alias='parasilo-22.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={}, net_devices=set())]}

In [9]:
# sync some more information in the host data structure (for illustration purpose here)
en.sync_info(roles, networks)

[WARNING]: No inventory was parsed, only implicit localhost is available



PLAY [all] ***********************************************************************************************************************************************************************************

TASK [hostname] ******************************************************************************************************************************************************************************
 [started TASK: hostname on parasilo-20.rennes.grid5000.fr]
 [started TASK: hostname on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]

PLAY [Gather facts for all hosts] ************************************************************************************************************************************************************

TASK [Gathering Facts] ***********************************************************************************************************************************************************************
ok: [parasilo-22.rennes.grid

ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed0:fd30/64
172.16.97.20/20
ip
::1/128
127.0.0.1/8
ip
fe80::eef4:bbff:fed1:60/64


## Acting on remote nodes

### run a command, filter results

In [10]:
results = en.run_command("whoami", roles=roles)
results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [whoami] ********************************************************************************************************************************************************************************
 [started TASK: whoami on parasilo-20.rennes.grid5000.fr]
 [started TASK: whoami on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:28.920997', 'end': '2021-08-27 10:01:28.929490', 'delta': '0:00:00.008493', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-22.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:28.928952', 'end': '2021-08-27 10:01:28.940732', 'delta': '0:00:00.011780', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})

In [11]:
one_result = results.filter(host=roles["control"][0].alias)[0]
one_result

CommandResult(host='parasilo-20.rennes.grid5000.fr', task='whoami', status='OK', payload={'cmd': 'whoami', 'stdout': 'root', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:28.920997', 'end': '2021-08-27 10:01:28.929490', 'delta': '0:00:00.008493', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'whoami', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['root'], 'stderr_lines': [], '_ansible_no_log': False})

In [12]:
one_result.payload["stdout"]

'root'

There are some specific shortcuts when the remote actions is a remote (shell) command: `.stdout`, `.stderr`, `.rc`

In [13]:
print(f"stdout = {one_result.stdout}\n", f"stderr={one_result.stderr}\n", f"return code = {one_result.rc}")

stdout = root
 stderr=
 return code = 0


### Filtering hosts on which the command is run

`run_command` acts on remote hosts. Those hosts can be given as a `Roles` type (output of `provider.init`) or as a list of `Host` or a single `Host`. 


In [14]:
# some roles
en.run_command("date", roles = roles)


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [date] **********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-20.rennes.grid5000.fr]
 [started TASK: date on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 10:01:29 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:29.257670', 'end': '2021-08-27 10:01:29.259796', 'delta': '0:00:00.002126', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 10:01:29 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-22.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 10:01:29 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:29.293038', 'end': '2021-08-27 10:01:29.295371', 'delta': '0:00:00.002333', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 10:01:29 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [15]:
# a list of hosts
en.run_command("date", roles = roles["control"])


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [date] **********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 10:01:29 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:29.603930', 'end': '2021-08-27 10:01:29.606062', 'delta': '0:00:00.002132', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 10:01:29 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [16]:
# a single host
en.run_command("date", roles=roles["control"][0])


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [date] **********************************************************************************************************************************************************************************
 [started TASK: date on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 10:01:29 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:29.927075', 'end': '2021-08-27 10:01:29.929257', 'delta': '0:00:00.002182', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 10:01:29 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

In [17]:
# you can forge some host yourself
# This should work if your SSH parameters are correct
en.run_command("date", roles=en.Host("rennes.grid5000.fr", user=en.g5k_api_utils.get_api_username()))


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [date] **********************************************************************************************************************************************************************************
 [started TASK: date on rennes.grid5000.fr]
changed: [rennes.grid5000.fr]


CommandResult(host='rennes.grid5000.fr', task='date', status='OK', payload={'cmd': 'date', 'stdout': 'Fri 27 Aug 2021 10:01:33 AM CEST', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:01:33.395495', 'end': '2021-08-27 10:01:33.431158', 'delta': '0:00:00.035663', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'date', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['Fri 27 Aug 2021 10:01:33 AM CEST'], 'stderr_lines': [], '_ansible_no_log': False})

### Remote actions

Tools like Ansible, Puppet, Chef, Terraform ... are shipped with a set of predefined remote actions to ease the administrator life.

Actions like copying file, adding some users, managing packages, making sure a line is absent from a configuration file, managing docker containers ... are first-class citizens actions and brings some nice garantees of correctness and idempotency.

There are 1000+ modules  available:
https://docs.ansible.com/ansible/2.9/modules/list_of_all_modules.html

---

EnOSlib wraps Ansible module and let you use them from Python (without writting any YAML file). You can call any module by using the `actions` context manager:

In the following we install docker (using g5k provided script) and a docker container. We also need to install the python docker binding on the remote machine so that Ansible can interact with the docker daemons on the remote machines. This block of actions is idempotent.


In [18]:
with en.actions(roles=roles) as a:
    # installing the docker daemon
    # prepending with a guard to make the command idempotent
    a.shell("which docker || /grid5000/code/bin/g5k-setup-docker")
    # install the python docker binding on the remote host
    # mandatory by the docker_container module
    a.pip(name="docker", state="present")
    # fire up a container (forward port 80 at the host level)
    a.docker_container(name="myserver", image="nginx", state="started", ports=["80:80"])
    # wait for the connection on the port 80 to be ready
    a.wait_for(port=80, state="started")
    # keep track of the result of each modules
    # not mandatory but nice :)
    results = a.results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [which docker || /grid5000/code/bin/g5k-setup-docker] ***********************************************************************************************************************************
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-20.rennes.grid5000.fr]
 [started TASK: which docker || /grid5000/code/bin/g5k-setup-docker on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]

TASK [pip] ***********************************************************************************************************************************************************************************
 [started TASK: pip on parasilo-20.rennes.grid5000.fr]
 [started TASK: pip on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.re

[DEPRECATION WARNING]: The container_default_behavior option will change its default value from "compatibility" to "no_defaults" in community.docker 2.0.0. To remove this warning, please 
specify an explicit value for it now. This feature will be removed from community.docker in version 2.0.0. Deprecation warnings can be disabled by setting deprecation_warnings=False in 
ansible.cfg.


changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]

TASK [wait_for] ******************************************************************************************************************************************************************************
 [started TASK: wait_for on parasilo-20.rennes.grid5000.fr]
 [started TASK: wait_for on parasilo-22.rennes.grid5000.fr]
ok: [parasilo-22.rennes.grid5000.fr]
ok: [parasilo-20.rennes.grid5000.fr]


In [ ]:
with en.actions(roles=roles) as a:
    # prepending with `which docker ||` makes this task idempotent
    a.shell("which docker || /grid5000/code/bin/g5k-setup-docker")
    a.pip(name="docker", state="present")
    a.docker_container(image="nginx", state="started", name="my_webserver")
    # we can capture the outputs if we really want to
    results = a.results
results

In [ ]:
results.filter(task="docker_container")[0]

### Background actions

Sometime you need to fire a process on some remote machines that needs to survive the remote connection that started it. EnOSlib provides a `keyword` argument for this purpose and can be used when calling modules (when supported).

In [21]:
# synchronous execution, will wait until the end of the shell command
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles)
results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] *************************************************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-20.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:02:22.212301', 'end': '2021-08-27 10:02:32.225737', 'delta': '0:00:10.013436', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-22.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'cmd': 'for i in $(seq 1 10); do sleep 1; echo toto; done', 'stdout': 'toto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto\ntoto', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:02:22.219646', 'end': '2021-08-27 10:02:32.241039', 'delta': '0:00:10.021393', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'for i in $(seq 1 10); do sleep 1; echo toto; done', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto', 'toto'], 'stderr_lines': [], '_ansible_no_log': False})

In [22]:
# The remote command will be daemonize on the remote machines
results = en.run_command("for i in $(seq 1 10); do sleep 1; echo toto; done", roles=roles, background=True)
results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [for i in $(seq 1 10); do sleep 1; echo toto; done] *************************************************************************************************************************************
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-20.rennes.grid5000.fr]
 [started TASK: for i in $(seq 1 10); do sleep 1; echo toto; done on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]


AsyncCommandResult(host='parasilo-20.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '637941494077.30972', 'results_file': '/root/.ansible_async/637941494077.30972', 'changed': True, '_ansible_no_log': False})
AsyncCommandResult(host='parasilo-22.rennes.grid5000.fr', task='for i in $(seq 1 10); do sleep 1; echo toto; done', status='OK', payload={'started': 1, 'finished': 0, 'ansible_job_id': '545447948435.15233', 'results_file': '/root/.ansible_async/545447948435.15233', 'changed': True, '_ansible_no_log': False})

In [25]:
# you can get back the status of the daemonized process by reading the remote results_file
h  = roles["control"][0]
result_file = results.filter(host=h.alias)[0].results_file
cat_result = en.run_command(f"cat {result_file}",roles=h)
cat_result


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [cat /root/.ansible_async/637941494077.30972] *******************************************************************************************************************************************
 [started TASK: cat /root/.ansible_async/637941494077.30972 on parasilo-20.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='cat /root/.ansible_async/637941494077.30972', status='OK', payload={'cmd': 'cat /root/.ansible_async/637941494077.30972', 'stdout': '{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-08-27 10:02:34.096318", "end": "2021-08-27 10:02:44.106273", "delta": "0:00:10.009955", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:02:44.221192', 'end': '2021-08-27 10:02:44.223592', 'delta': '0:00:00.002400', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'cat /root/.ansible_async/637941494077.30972', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['{"cmd": "for i in $(seq 1 10); do sleep 1; echo toto; done", "stdout": "toto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto\\ntoto", "stderr": "", "rc": 0, "start": "2021-08-27 10:02:34.096318", "end": "2021-08-27 10:02:44.106273", "delta": "0:00:10.009955", "changed": true, "invocation": {"module_args": {"_raw_params": "for i in $(seq 1 10); do sleep 1; echo toto; done", "_uses_shell": true, "warn": true, "stdin_add_newline": true, "strip_empty_ends": true, "argv": null, "chdir": null, "executable": null, "creates": null, "removes": null, "stdin": null}}}'], 'stderr_lines': [], '_ansible_no_log': False})

In [26]:
# the result_file content is json encoded so decoding it
import json
print(json.loads(cat_result[0].stdout)["stdout"])

toto
toto
toto
toto
toto
toto
toto
toto
toto
toto


## Using variables

### Same variable value for everyone

Nothing surprising here, you can use regular python interpolation (e.g a `f-string`).
String are interpolated by the interpreter before being manipulated.

In [27]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command(f"ping -c 5 {host_to_ping}", roles=roles)
results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [ping -c 5 parasilo-20.rennes.grid5000.fr] **********************************************************************************************************************************************
 [started TASK: ping -c 5 parasilo-20.rennes.grid5000.fr on parasilo-20.rennes.grid5000.fr]
 [started TASK: ping -c 5 parasilo-20.rennes.grid5000.fr on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='ping -c 5 parasilo-20.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-20.rennes.grid5000.fr', 'stdout': 'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.041 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.031 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.017 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.032 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.017 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 85ms\nrtt min/avg/max/mdev = 0.017/0.027/0.041/0.010 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:02:49.579346', 'end': '2021-08-27 10:02:53.677186', 'delta': '0:00:04.097840', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-20.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.041 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.031 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.017 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.032 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.017 ms', '', '--- parasilo-20.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 85ms', 'rtt min/avg/max/mdev = 0.017/0.027/0.041/0.010 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-22.rennes.grid5000.fr', task='ping -c 5 parasilo-20.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-20.rennes.grid5000.fr', 'stdout': 'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.312 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.197 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.189 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.169 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.154 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 102ms\nrtt min/avg/max/mdev = 0.154/0.204/0.312/0.056 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:02:49.604515', 'end': '2021-08-27 10:02:53.717385', 'delta': '0:00:04.112870', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-20.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.312 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.197 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.189 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.169 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl

In [28]:
[(r.host, r.stdout) for r in results]

[('parasilo-20.rennes.grid5000.fr',
  'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.041 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.031 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.017 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.032 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.017 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 85ms\nrtt min/avg/max/mdev = 0.017/0.027/0.041/0.010 ms'),
 ('parasilo-22.rennes.grid5000.fr',
  'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.312 ms\n64 bytes from parasilo-20.rennes.grid

### Using templates / Ansible variables

There's an alternative way to pass a variable to a task: using `extra_vars`.
The difference with the previous case (python interpreted variables) is the fact that the variable is interpolated right before execution happens on the remote node.
One could imagine the the value is broadcasted to all nodes and replaced right before the execution.

To indicate that we want to use this kind of variables, we need to pass its value using the `extra_vars` dictionnary and use a template (`{{ ... }}`) in the task description.

In [29]:
host_to_ping = roles["control"][0].alias
host_to_ping

results = en.run_command("ping -c 5 {{ my_template_variable }}", roles=roles, extra_vars=dict(my_template_variable=host_to_ping))
results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [ping -c 5 parasilo-20.rennes.grid5000.fr] **********************************************************************************************************************************************
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-20.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ my_template_variable }} on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]


CommandResult(host='parasilo-20.rennes.grid5000.fr', task='ping -c 5 parasilo-20.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-20.rennes.grid5000.fr', 'stdout': 'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.040 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.031 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.032 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.017 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.032 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 81ms\nrtt min/avg/max/mdev = 0.017/0.030/0.040/0.008 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:03:03.324193', 'end': '2021-08-27 10:03:07.405223', 'delta': '0:00:04.081030', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-20.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.040 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.031 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.032 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.017 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.032 ms', '', '--- parasilo-20.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 81ms', 'rtt min/avg/max/mdev = 0.017/0.030/0.040/0.008 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-22.rennes.grid5000.fr', task='ping -c 5 parasilo-20.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-20.rennes.grid5000.fr', 'stdout': 'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.189 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.198 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.198 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.174 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.151 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 108ms\nrtt min/avg/max/mdev = 0.151/0.182/0.198/0.017 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:03:03.368123', 'end': '2021-08-27 10:03:07.477422', 'delta': '0:00:04.109299', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-20.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.189 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.198 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.198 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.174 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl

### Host specific variables

In the above, we've seen how a common value can be broadcasted to all remote nodes.  What if we want host specific value ?

For instance in our case we'd like `host 1` to ping `host 2` and `host 2` to ping `host 1`. That make the `host_to_ping` variable host-specific.

For this purpose you can use the `extra` attribute of the `Host` objects and use a template as before.

In [33]:
control_host = roles["control"][0]
compute_host = roles["compute"][0]
control_host.extra.update(host_to_ping=compute_host.address)
compute_host.extra.update(host_to_ping=control_host.address)
control_host

Host(address='parasilo-20.rennes.grid5000.fr', alias='parasilo-20.rennes.grid5000.fr', user='root', keyfile=None, port=None, extra={'host_to_ping': 'parasilo-22.rennes.grid5000.fr'}, net_devices=set())

> Note that the `extra` attribute is mutable :(

In [31]:
results = en.run_command("ping -c 5 {{ host_to_ping }}", roles=roles)
results


PLAY [all] ***********************************************************************************************************************************************************************************

TASK [ping -c 5 parasilo-22.rennes.grid5000.fr] **********************************************************************************************************************************************
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-20.rennes.grid5000.fr]
 [started TASK: ping -c 5 {{ host_to_ping }} on parasilo-22.rennes.grid5000.fr]
changed: [parasilo-22.rennes.grid5000.fr]
changed: [parasilo-20.rennes.grid5000.fr]


CommandResult(host='parasilo-22.rennes.grid5000.fr', task='ping -c 5 parasilo-20.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-20.rennes.grid5000.fr', 'stdout': 'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.141 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.139 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.120 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.138 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.177 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 104ms\nrtt min/avg/max/mdev = 0.120/0.143/0.177/0.018 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:03:07.884501', 'end': '2021-08-27 10:03:11.989088', 'delta': '0:00:04.104587', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-20.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.141 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.139 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.120 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.138 ms', '64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.177 ms', '', '--- parasilo-20.rennes.grid5000.fr ping statistics ---', '5 packets transmitted, 5 received, 0% packet loss, time 104ms', 'rtt min/avg/max/mdev = 0.120/0.143/0.177/0.018 ms'], 'stderr_lines': [], '_ansible_no_log': False})
CommandResult(host='parasilo-20.rennes.grid5000.fr', task='ping -c 5 parasilo-22.rennes.grid5000.fr', status='OK', payload={'cmd': 'ping -c 5 parasilo-22.rennes.grid5000.fr', 'stdout': 'PING parasilo-22.rennes.grid5000.fr (172.16.97.22) 56(84) bytes of data.\n64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=1 ttl=64 time=0.119 ms\n64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=2 ttl=64 time=0.183 ms\n64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=3 ttl=64 time=0.171 ms\n64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=4 ttl=64 time=0.061 ms\n64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=5 ttl=64 time=0.091 ms\n\n--- parasilo-22.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 108ms\nrtt min/avg/max/mdev = 0.061/0.125/0.183/0.046 ms', 'stderr': '', 'rc': 0, 'start': '2021-08-27 10:03:07.872591', 'end': '2021-08-27 10:03:11.981287', 'delta': '0:00:04.108696', 'changed': True, 'invocation': {'module_args': {'_raw_params': 'ping -c 5 parasilo-22.rennes.grid5000.fr', '_uses_shell': True, 'warn': True, 'stdin_add_newline': True, 'strip_empty_ends': True, 'argv': None, 'chdir': None, 'executable': None, 'creates': None, 'removes': None, 'stdin': None}}, 'stdout_lines': ['PING parasilo-22.rennes.grid5000.fr (172.16.97.22) 56(84) bytes of data.', '64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=1 ttl=64 time=0.119 ms', '64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=2 ttl=64 time=0.183 ms', '64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=3 ttl=64 time=0.171 ms', '64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=4 ttl=64 time=0.061 ms', '64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=5 t

In [32]:
[(r.host, r.stdout) for r in results]

[('parasilo-22.rennes.grid5000.fr',
  'PING parasilo-20.rennes.grid5000.fr (172.16.97.20) 56(84) bytes of data.\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=1 ttl=64 time=0.141 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=2 ttl=64 time=0.139 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=3 ttl=64 time=0.120 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=4 ttl=64 time=0.138 ms\n64 bytes from parasilo-20.rennes.grid5000.fr (172.16.97.20): icmp_seq=5 ttl=64 time=0.177 ms\n\n--- parasilo-20.rennes.grid5000.fr ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 104ms\nrtt min/avg/max/mdev = 0.120/0.143/0.177/0.018 ms'),
 ('parasilo-20.rennes.grid5000.fr',
  'PING parasilo-22.rennes.grid5000.fr (172.16.97.22) 56(84) bytes of data.\n64 bytes from parasilo-22.rennes.grid5000.fr (172.16.97.22): icmp_seq=1 ttl=64 time=0.119 ms\n64 bytes from parasilo-22.rennes.gri

## Cleaning

In [34]:
provider.destroy()

INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lille
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from luxembourg
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from lyon
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nancy
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from nantes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading 1816344 from rennes
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Reloading rsd-01 from sophia
INFO:enoslib.infra.enos_g5k.g5k_api_utils:Killing the job (rennes, 1816344)
